<a href="https://colab.research.google.com/github/dudeurv/SAM_MRI/blob/main/An_Introduction_to_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

train_df = pd.read_csv('sample_data/california_housing_train.csv')
test_df = pd.read_csv('sample_data/california_housing_test.csv')

train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [ ]:
X_train_np = train_df.to_numpy()[:, :-1]
y_train_np = train_df.to_numpy()[:, -1]

X_train_np.shape, y_train_np.shape

((17000, 8), (17000,))

In [ ]:
X_test_np = test_df.to_numpy()[:, :-1]
y_test_np = test_df.to_numpy()[:, -1]

X_test_np.shape, y_test_np.shape

((3000, 8), (3000,))

In [ ]:
import torch
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(torch.tensor(X_train_np, dtype=torch.float),
                              torch.tensor(y_train_np.reshape((-1, 1)), dtype=torch.float))

train_dataset

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=128)

for X, y in train_dataloader:
  print(X.shape, y.shape)
  break

torch.Size([128, 8]) torch.Size([128, 1])


In [ ]:
test_dataset = TensorDataset(torch.tensor(X_test_np, dtype=torch.float),
                             torch.tensor(y_test_np.reshape((-1, 1)), dtype=torch.float))

test_dataset

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64)

for X, y in test_dataloader:
  print(X.shape, y.shape)
  break

torch.Size([64, 8]) torch.Size([64, 1])


In [ ]:
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()

    self.hidden_layer_1 = nn.Linear(8, 64)
    self.hidden_activation = nn.ReLU()

    self.out = nn.Linear(64, 1)

  def forward(self, x):
    x = self.hidden_layer_1(x)
    x = self.hidden_activation(x)
    x = self.out(x)
    return x

model = NeuralNet().to(device)
print(model)

Using cpu device
NeuralNet(
  (hidden_layer_1): Linear(in_features=8, out_features=64, bias=True)
  (hidden_activation): ReLU()
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  model.train()
  train_loss = 0

  for i, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_hat = model(X)
    mse = loss_fn(y_hat, y)
    train_loss += mse.item()

    optimizer.zero_grad()
    mse.backward()
    optimizer.step()

  num_batches = len(dataloader)
  train_mse = train_loss / num_batches
  print(f'Train RMSE: {train_mse**(1/2)}')

In [ ]:
def test(dataloader, model, loss_fn):
  model.eval()
  test_loss = 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      y_hat = model(X)
      test_loss += loss_fn(y_hat, y).item()

  num_batches = len(dataloader)
  test_mse = test_loss / num_batches

  print(f'Test RMSE: {test_mse**(1/2)}\n')

In [ ]:
epochs = 10

for epoch in range(epochs):
  print(f"Epoch {epoch+1}:")

  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

Epoch 1:
Train RMSE: 184241.32032083333
Test RMSE: 163106.84380073787

Epoch 2:
Train RMSE: 167047.43746297748
Test RMSE: 156332.1380509282

Epoch 3:
Train RMSE: 162048.42029189222
Test RMSE: 154622.02344798407

Epoch 4:
Train RMSE: 158527.8656875776
Test RMSE: 148765.38011358836

Epoch 5:
Train RMSE: 152712.52059496025
Test RMSE: 144312.98812045754

Epoch 6:
Train RMSE: 146451.2233031102
Test RMSE: 139743.99501093783

Epoch 7:
Train RMSE: 139396.17572126677
Test RMSE: 135626.48756050566

Epoch 8:
Train RMSE: 132340.37894182623
Test RMSE: 134344.56421060054

Epoch 9:
Train RMSE: 126444.917672372
Test RMSE: 128813.5572724839

Epoch 10:
Train RMSE: 121249.20942292453
Test RMSE: 125605.4198911916

